In [ ]:
638*3*70*30 / 1e3

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('tableau-colorblind10')
#%matplotlib widget

In [ ]:
from scipy.signal import welch, get_window
from obspy.signal.filter import bandpass

from obspy.clients.filesystem.sds import Client
from obspy.clients.fdsn import RoutingClient
from obspy.core import UTCDateTime as UTC
from obspy.signal import util

In [ ]:
import obspy

In [ ]:
obspy.__version__

In [ ]:
plt.style.available

In [ ]:
now = UTC()

In [ ]:
now.julday

In [ ]:
sdsclient = Client('/home/lehr/sds/data')

In [ ]:
sdsclient.get_all_stations()

In [ ]:
sdsclient.get_all_nslc()

In [ ]:
for net, sta, loc, cha in sdsclient.get_all_nslc():
    print(net, sta, loc, cha)

In [ ]:
network = 'GR'
station = 'BFO'
location = ''
channel = 'HHZ'
date = UTC("2020-04-17")
starttime = date-3600
endtime = starttime + 26*3600
st = sdsclient.get_waveforms(network, station, location, channel,starttime, endtime)

In [ ]:
%matplotlib widget
st.plot(show=False)

In [ ]:
st.merge()

In [ ]:
st[0].stats

In [ ]:
rc = RoutingClient('eida-routing')

In [ ]:
network = 'GR'
station = 'BFO'
location = ''
channel = '*Z'
starttime = UTC("2020-01-01")
endtime = UTC("2021-01-01")
level = "response"
inv = rc.get_stations(network=network, station=station, location=location, starttime=starttime, endtime=endtime, channel=channel, level=level)

In [ ]:
inv.get_contents()

In [ ]:
seedid = 'GR.BFO..HHZ'

In [ ]:
inv.get_channel_metadata(seedid)

In [ ]:
inv.get_response(seedid, starttime)

In [ ]:
st = rc.get_waveforms(network=network, station=station, location=location, starttime=starttime, endtime=endtime, channel=channel)

In [ ]:
%matplotlib widget
st.plot(show=False)

# Processing

In [ ]:
st.taper(0.1, max_length=2700)

In [ ]:
st.plot(show=False)

## Spectrum over whole day

In [ ]:
#tr = st[0]
lw = 4.
plt.figure(figsize=(10, 5))
N = 2048
for tr in st:
    #for N in [512,1024, 2048]:
    freq, P = welch(tr.data, tr.stats.sampling_rate, nperseg=N, nfft=1*N)
    plt.plot(freq, P, lw=lw, alpha=0.95)
    #lw-=1
    #plt.xlim(2, 9)
#plt.xlim(4, 14)
plt.yscale('log')

In [ ]:
st.filter('bandpass', freqmin=4, freqmax=14)

In [ ]:
st.plot(show=False)

# Noise level

In [ ]:
tr = st[0]

In [ ]:
tr

In [ ]:
T = 3600
sr = tr.stats.sampling_rate
nwin = int(T * sr)
win = get_window('boxcar', nwin, fftbins=False)

In [ ]:
tr.data.size, nwin

In [ ]:
matrix, nx, ny  = util.enframe(tr.data, win, win.size)
matrix = matrix[1:-1]

In [ ]:
matrix.shape, ny, nx

In [ ]:
prctl = np.percentile(matrix, 75, axis=1)

In [ ]:
prctl.shape

In [ ]:
plt.plot(prctl)

In [ ]:
prctl_man = []
for r in matrix:
    r = np.sort(r.copy())
    i = int(r.size*0.75)
    prctl_man.append(r[i+1])
    
plt.plot(prctl_man)
plt.plot(prctl)
    

## Spectrum

In [ ]:
freq, P = welch(matrix, fs=sr, nperseg=2048, axis=1)

In [ ]:
P.shape, freq.shape

In [ ]:
plt.pcolor(freq, np.arange(24), P)

# Instrument response

In [ ]:
network = 'GR'
station = 'BFO'
location = ''
channel = '*Z'
starttime = UTC("2020-01-01")
endtime = UTC("2021-01-01")


In [ ]:
level = "response"
rc = RoutingClient('eida-routing')
inv = rc.get_stations(network=network, station=station, location=location, starttime=starttime, endtime=endtime, channel=channel, level=level)

In [ ]:
seedid = 'GR.BFO..HHZ'

In [ ]:
resp = inv.get_response(seedid, starttime)

In [ ]:
resp.instrument_sensitivity.value

In [ ]:
#starttime = startdate
endtime = starttime + 24*3600
print(starttime, endtime)
st = sdsclient.get_waveforms(network, station, 
                         location, channel,starttime, endtime)

In [ ]:
st.plot(show=False)

In [ ]:
st.remove_sensitivity(inv)

In [ ]:
st.plot(show=False)

In [ ]:
st.detrend('constant')

In [ ]:
st.plot(show=False)

# Mass production

In [ ]:
def process(tr, winlen_in_s, nperseg, fmin, fmax):
    
    # Convert data into matrix (n_slices, npts)
    sr = tr.stats.sampling_rate
    nwin = int(winlen_in_s * sr)
    win = get_window('boxcar', nwin, fftbins=False)
    
    matrix, nx, ny  = enframe(tr.data, win, win.size)
    #matrix[matrix==-99999] = np.nan
    print(matrix.argmin(), matrix.min(), matrix.dtype)
    if np.nan in matrix:
        print("Found nans")
    #matrix = matrix[1:-1]
    freq, P = welch(matrix, fs=sr, nperseg=nperseg, axis=1)
    
    for i, r in enumerate(matrix):
        matrix[i,:] = bandpass(r, fmin, fmax, sr)
    
    prctl = np.nanpercentile(matrix, 25, axis=1)
    
    
    return prctl, freq, P

In [ ]:
network = 'GR'
station = 'BFO'
location = ''
channel = 'HHZ'
startdate = UTC("2020-04-16")
enddate = UTC("2020-04-19")
overlap = 0 #3600

fmin, fmax = (4, 14)
nperseg = 2048
winlen_in_s = 3600

AMP = []
PXX = []
starttime = startdate-overlap
inv = rc.get_stations(network=network, station=station, 
                     location=location, starttime=startdate, 
                     endtime=enddate, channel=channel, 
                     level='response')
while starttime < enddate - overlap:
    
    endtime = starttime + 24*3600+2*overlap
    print(starttime, endtime)
    st = sdsclient.get_waveforms(network, station, 
                             location, channel,starttime, endtime)
    st.remove_sensitivity(inv)
    #st.detrend('constant')
    st.merge(fill_value=np.nan)
    print(st[0])
    st.trim(starttime, endtime, pad=True, fill_value=np.nan)
   
    if len(st) > 1:
        raise RuntimeWarning("More than 1 trace in stream!")
    tr = st[0]
    #if np.any(np.isnan(tr.data)):
    #    break
    print(tr)
    amp, freq, pxx = process(tr, winlen_in_s, nperseg, fmin, fmax)
    print(amp.shape, pxx.shape)
    if -99999 in tr.data or amp.size != 24:
        break
    AMP.append(amp[1:-1])
    PXX.append(pxx[1:-1,:])
    
    starttime = starttime + 24*3600
    

In [ ]:
AMP

In [ ]:
AMP

In [ ]:
st = sdsclient.get_waveforms(network, station, 
                             location, channel,starttime, endtime)

In [ ]:
st.remove_sensitivity(inv)

In [ ]:
st.plot(show=False)

In [ ]:
st.merge(fill_value=np.nan)

In [ ]:
st.plot(show=False)

In [ ]:
st.trim(starttime, endtime, pad=True, fill_value=np.nan)

In [ ]:
st.plot(show=False)

In [ ]:
st.detrend('constant')

In [ ]:
st.plot(show=False)

In [ ]:
sr = tr.stats.sampling_rate
nwin = int(winlen_in_s * sr)
win = get_window('boxcar', nwin, fftbins=False)
#win = np.asarray(win, dtype=np.int32)
matrix, nx, ny  = enframe(tr.data, win, win.size)

In [ ]:
matrix.argmin(), matrix.min(), matrix.dtype

Why is the minimum not -99999? Boxcar not a proper boxcar?

In [ ]:
matrix[matrix==-99999]

In [ ]:
x = np.int32(-99999)

In [ ]:
np.float64(x)

In [ ]:
tr.data.dtype

In [ ]:
matrix.shape

In [ ]:
matrix.dtype

In [ ]:
tr.plot(show=False, starttime=UTC("2020-04-15T23:59"), endtime=UTC("2020-04-16T00:01"))

In [ ]:
# %load -n util.enframe
#from scipy import fix
def enframe(x, win, inc):
    """
    Splits the vector up into (overlapping) frames beginning at increments
    of inc. Each frame is multiplied by the window win().
    The length of the frames is given by the length of the window win().
    The centre of frame I is x((I-1)*inc+(length(win)+1)/2) for I=1,2,...

    :param x: signal to split in frames
    :param win: window multiplied to each frame, length determines frame length
    :param inc: increment to shift frames, in samples
    :return f: output matrix, each frame occupies one row
    :return length, no_win: length of each frame in samples, number of frames
    """
    nx = len(x)
    nwin = len(win)
    if (nwin == 1):
        length = win
    else:
        # length = next_pow_2(nwin)
        length = nwin
    nf = int(np.fix((nx - length + inc) // inc))
    # f = np.zeros((nf, length))
    indf = inc * np.arange(nf)
    inds = np.arange(length) + 1
    f = x[(np.transpose(np.vstack([indf] * length)) +
           np.vstack([inds] * nf)) - 1]
    if (nwin > 1):
        w = np.transpose(win)
        f = f * np.vstack([w] * nf)
    #f = signal.detrend(f, type='constant')
    no_win, _ = f.shape
    return f, length, no_win

In [ ]:
AMP = np.array(AMP)
PXX = np.array(PXX)

In [ ]:
startdate.format_seedlink()

In [ ]:
"data/{}.{}.{}.{}_{}-{}_AMP.npy".format(network, station, location, channel, startdate.date, enddate.date)

In [ ]:
np.save("data/{}.{}.{}.{}_{}_{}_AMP.npy".format(network, station, location, channel, startdate, enddate), AMP)

In [ ]:
np.save("data/{}.{}.{}.{}_{}_{}_PXX.npy".format(network, station, location, channel, startdate, enddate), PXX)

In [ ]:
import plotly.graph_objects as go

In [ ]:
z = AMP #np.clip(AMP, None,  a_max=5.5)
sh_0, sh_1 = z.shape
y, x = np.linspace(0, sh_0-1, sh_0), np.linspace(0, sh_1-1, sh_1)
fig = go.Figure(data=[go.Surface(z=z, x=x, y=y, name='amplitude', cmin=2, cmax=None)])
fig.update_layout(title='75%-amplitude', autosize=True,
                  width=800, height=500,
                  scene=dict(aspectmode='manual', aspectratio=dict(x=1, y=2, z=0.5))
                  #margin=dict(l=65, r=50, b=65, t=90)
                 )
fig.show()

In [ ]:
0.75*39

In [ ]:
from matplotlib import cbook
from matplotlib import cm
from matplotlib.colors import LightSource

In [ ]:
z = AMP.copy()

nrows, ncols = z.shape
x = np.linspace(0, 23, ncols)
y = np.linspace(0, nrows-1, nrows)
x, y = np.meshgrid(x, y)

region = np.s_[5:50, 5:50]
x, y, z = x[region], y[region], z[region]

# Set up plot
fig, ax = plt.subplots(subplot_kw=dict(projection='3d'))

ls = LightSource(270, 45)
# To use a custom hillshading mode, override the built-in shading and pass
# in the rgb colors of the shaded surface calculated from "shade".
rgb = ls.shade(z, cmap=cm.inferno, vert_exag=5, blend_mode='soft')
surf = ax.plot_surface(x, y, z, rstride=1, cstride=1, facecolors=rgb,
                       linewidth=0, antialiased=False, shade=False)


In [ ]:
%%time?